In [4]:
import glob
import gzip
import json
import os
import time
from datetime import datetime, date
import pickle
from functools import reduce
import random
import re

def get_single_article_string(article):
        
    curr_title=article['title']
    abstract_inverted_index = article['abstract_inverted_index']

    # Flatten the inverted index into a list of (position, word) tuples
    #position_word_list = [(position, word) for word, positions in abstract_inverted_index.items() for position in positions]
    ###############修改代码####################
    position_word_list = [(position, word) for word, positions in (abstract_inverted_index or {}).items() for position in positions]
    # Sort the list by position and extract the words
    sorted_abstract = sorted(position_word_list)
    curr_abstract = ' '.join(word for position, word in sorted_abstract)

    # Replace strings according to the replace_pairs list
    replace_pairs=[['\n',' '],['-',' '],[' \" a','oa'],['\" a','ae'],['\"a','ae'],[' \" o','oe'],['\" o','oe'],['\"o','oe'],[' \" u','ue'],['\" u','ue'],['\"u','ue'],[' \' a','a'],[' \' e','e'],[' \' o','o'],["\' ", ""],["\'", ""],['  ',' '],['  ',' ']]

    article_string=(curr_title +' '+ curr_abstract).lower()
    article_string = reduce(lambda text, pair: text.replace(pair[0], pair[1]), replace_pairs, article_string)

    return article_string

# Define a sorting key function to extract the date and part number from the path
def get_date_and_part_from_path(path):
    date_folder = os.path.dirname(path)
    date_str = date_folder.split('=')[-1]

    file_name = os.path.basename(path)
    part_str = file_name.split('_')[-1].split('.')[0]

    return date_str, int(part_str)


def extract_id(filename):
    match = re.search(r'log_edge_part_(\d+)_', filename)
    if match:
        return int(match.group(1))
    return None


# define a log foler
log_folder = 'logs_pair'
# define edge_list foler
edge_folder = 'concept_pair'
edge_folder_log = 'concept_pair_log'

try:
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)
    
    if not os.path.exists(edge_folder):
        os.makedirs(edge_folder)
        
    if not os.path.exists(edge_folder_log):
        os.makedirs(edge_folder_log)

except FileExistsError:
    pass

data_folder="data_concept_graph" # the folder that contain the concept file
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
concept_folder = os.path.join(parent_dir, data_folder)


# Define the base folder, date pattern and file pattern
#project_path="/u/xmgu/projects/semnet_openalex" # change to your path
#base_folder=os.path.join(project_path,'openalex_workdata_filtered/data/works/')
##############已修改路径#######################
base_folder = 'D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works/'
date_pattern = 'updated_date=*'
#######################已修改文件名#####################
file_pattern = 'part_*.gz'

# Find all the files matching the pattern
file_paths = glob.glob(f'{base_folder}/{date_pattern}/{file_pattern}')
# Sort the file_paths list in ascending order based on the date and part number
file_paths = sorted(file_paths, key=get_date_and_part_from_path)
############修改日期##################
# Define the date range or specific folders to include
start_date = datetime.strptime("2022-12-20", "%Y-%m-%d")
end_date = datetime.strptime("2025-10-01", "%Y-%m-%d")

# Filter the file_paths list based on the date range or specific folders
curr_run_file_paths = [path for path in file_paths if start_date <= datetime.strptime(get_date_and_part_from_path(path)[0], "%Y-%m-%d") <= end_date]

# Read all concepts from full_final_concepts/full_domain_concept.txt
concepts_files = os.path.join(concept_folder, 'full_domain_concept.txt')
with open(concepts_files, 'r') as file:
    full_concepts = [concept.strip() for concept in file.readlines()]

# Define a list to store the edge lists
paper_starting_date = date(1990,1,1) 
write_file=0

rnd_time=random.random()*60
time.sleep(rnd_time)

while write_file <=len(curr_run_file_paths): # curr_run_file_paths
    
    curr_ID = random.randint(0, len(curr_run_file_paths)-1) # get a random number between 0 and the number of files

    formatted_ID = '{:03d}'.format(curr_ID)

    edge_file=os.path.join(edge_folder, 'edge_part_'+formatted_ID+'.gz')
    edge_file_log=os.path.join(edge_folder_log, 'edge_part_'+formatted_ID+'.txt')
    log_file_txt=os.path.join(log_folder, 'log_edge_part_'+formatted_ID+'.txt')

    log_file_txt_finish=os.path.join(log_folder, 'log_edge_part_'+formatted_ID+'_finish.txt')
    log_file_txt_empty=os.path.join(log_folder, 'log_edge_part_'+formatted_ID+'_empty.txt')

    if not os.path.exists(log_file_txt):
        current_time=datetime.now()
        open(log_file_txt, 'a').close()
            
        file_path=curr_run_file_paths[curr_ID]
        with open(log_file_txt, 'a') as log_file:
            log_file.write(f'Current time: {current_time}; Number of files: {len(curr_run_file_paths)}; Number of concepts: {len(full_concepts)}\n\n')
            log_file.write(f'Start the File: {file_path}; Current time: {datetime.now()} \n\n')
            
        with gzip.open(file_path, 'rt') as file:
            lines = file.readlines()

            if not lines: # if lines is not empty
                print(f'File {file_path} is empty')
                write_file+=1
                with open(log_file_txt_empty, 'a') as log_file:
                    log_file.write(f'Current File: {file_path}; Paper: {len(lines)}; File is Empty!\n')
                
            else:
                edge_lists=[]
                for id_line, line in enumerate(lines):
                    time_start_line=time.time()

                    article_object = json.loads(line) # Load the JSON object
                    get_date = datetime.strptime(article_object['publication_date'], "%Y-%m-%d").date()
                    curr_paper_time = (get_date - paper_starting_date).days
                    curr_all_citations=article_object['cited_by_count']
                    curr_citations_per_year=article_object['counts_by_year']
                    curr_article=get_single_article_string(article_object)

                    # Check if the article contains any of the concepts
                    concepts_for_single_paper=[]
                    for id_concept, concept in enumerate(full_concepts):
                        if concept in curr_article: # if the paper contains the concept; then store its concept index 
                            concepts_for_single_paper.append(id_concept)

                    for ii in range(len(concepts_for_single_paper)):
                        for jj in range(ii+1,len(concepts_for_single_paper)):
                            edge_lists.append([concepts_for_single_paper[ii],concepts_for_single_paper[jj],curr_paper_time,curr_all_citations,curr_citations_per_year])
                    

                    if id_line % 10000 == 0:
                        with open(log_file_txt, 'a') as log_file:
                            log_file.write(f'Current File: {file_path}; Paper: {len(lines)}; Processed: {(id_line+1)/len(lines)}; time: {time.time()-time_start_line}\n')

                # Finish the current file, then store edge_lists to a pickle file
                with gzip.open(edge_file, 'wb') as output_file:
                    pickle.dump(edge_lists, output_file)
                    write_file+=1
                    
                with open(edge_file_log, 'a') as log_file:
                    log_file.write(f'\nedge_list={len(edge_lists)}')
                
                with open(log_file_txt, 'a') as log_file:
                    log_file.write(f'\n\nFinish Time: {datetime.now()}; Current File: {file_path}; Processed: {write_file}/{len(curr_run_file_paths)}, i.e., {write_file/len(curr_run_file_paths)} \n')
                    log_file.write(f'\nedge_list: {len(edge_lists)}\n')
                
                with open(log_file_txt_finish, 'a') as log_file:
                    log_file.write(f'\n\nFinish Time: {datetime.now()}; Current File: {file_path} \n')
                    
                rnd_time=random.random()*5
                time.sleep(rnd_time)
                
    else:
        finish_pattern = os.path.join(log_folder, 'log_edge_part_*_finish.txt')
        empty_pattern = os.path.join(log_folder, 'log_edge_part_*_empty.txt')
        finished_files = [f for f in glob.glob(finish_pattern) if extract_id(f) in range(0, len(curr_run_file_paths))]
        empty_files = [f for f in glob.glob(empty_pattern) if extract_id(f) in range(0, len(curr_run_file_paths))]

        # Count files that match each pattern
        total_files = len(finished_files) + len(empty_files)

        # Check if the total count is larger than 391
        if total_files >= len(curr_run_file_paths):
            print(f"{datetime.now()}:Finish run!")
            break



with open("job_finish.txt", 'a') as f:
    f.write(f'\nFinish all: {datetime.now()}\n')




TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

ai增加调试， 处理 abstract_inverted_index 为 None 的情况

In [10]:
! pip install pyahocorasick


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


从按日期分片的论文数据文件中读取文献信息，使用Aho-Corasick自动机算法高效匹配预先定义的领域概念词典，为每篇论文中同时出现的概念对生成带权重的边数据（包含概念索引、论文时间戳、引用数量和年度引用信息）。系统采用分批处理和断点续传机制，通过进度文件记录处理状态，将生成的边数据压缩保存，同时输出详细的处理日志，最终形成完整的概念关联网络基础数据集。
构建包含失败指针的Trie树，能够一次性在文本中扫描匹配成千上万个预定义模式串。只需扫描文本一次就能找出所有匹配的模式，时间复杂度为O(n+m)，远优于传统逐个匹配的O(n×m)方法。

edge_lists.append([
    concepts_for_single_paper[ii],      # 概念A的ID
    concepts_for_single_paper[jj],      # 概念B的ID  
    curr_paper_time,                    # 论文发表时间（天数）
    curr_all_citations,                 # 总引用次数
    curr_citations_per_year             # 年度引用统计
])

In [12]:
import glob
import gzip
import json
import os
import time
import pickle
import random
import re
from datetime import datetime, date
from functools import reduce
import ahocorasick  # pip install pyahocorasick
import json

# -----------------------------
# 函数定义
# -----------------------------
def get_single_article_string(article):
    # 安全地获取标题和摘要
    curr_title = article.get('title') or ''
    abstract_inverted_index = article.get('abstract_inverted_index') or {}

    # 展平 inverted index
    position_word_list = [
        (position, word)
        for word, positions in abstract_inverted_index.items()
        for position in positions
    ]

    # 组装摘要文本
    if position_word_list:
        sorted_abstract = sorted(position_word_list)
        curr_abstract = ' '.join(word for position, word in sorted_abstract)
    else:
        curr_abstract = ''

    # 清理特殊符号
    replace_pairs = [
        ['\n', ' '], ['-', ' '], [' \" a', 'oa'], ['\" a', 'ae'], ['\"a', 'ae'],
        [' \" o', 'oe'], ['\" o', 'oe'], ['\"o', 'oe'], [' \" u', 'ue'], ['\" u', 'ue'],
        ['\"u', 'ue'], [' \' a', 'a'], [' \' e', 'e'], [' \' o', 'o'],
        ["\' ", ""], ["\'", ""], ['  ', ' '], ['  ', ' ']
    ]

    # 拼接标题 + 摘要
    article_string = (curr_title + ' ' + curr_abstract).lower()
    article_string = reduce(lambda text, pair: text.replace(pair[0], pair[1]), replace_pairs, article_string)

    return article_string



def get_date_and_part_from_path(path):
    date_folder = os.path.dirname(path)
    date_str = date_folder.split('=')[-1]
    file_name = os.path.basename(path)
    part_str = file_name.split('_')[-1].split('.')[0]
    return date_str, int(part_str)


def extract_id(filename):
    match = re.search(r'log_edge_part_(\d+)_', filename)
    if match:
        return int(match.group(1))
    return None


# -----------------------------
# 路径定义
# -----------------------------
log_folder = 'logs_pair'
edge_folder = 'concept_pair'
edge_folder_log = 'concept_pair_log'
data_folder = "data_concept_graph"
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
concept_folder = os.path.join(parent_dir, data_folder)
progress_file = "progress.json"

for folder in [log_folder, edge_folder, edge_folder_log]:
    os.makedirs(folder, exist_ok=True)

# -----------------------------
# 文件路径筛选
# -----------------------------
base_folder = 'D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works/'
file_paths = glob.glob(f'{base_folder}/updated_date=*/part_*.gz')
file_paths = sorted(file_paths, key=get_date_and_part_from_path)

start_date = datetime.strptime("2022-12-20", "%Y-%m-%d")
end_date = datetime.strptime("2025-10-01", "%Y-%m-%d")

curr_run_file_paths = [
    path for path in file_paths
    if start_date <= datetime.strptime(get_date_and_part_from_path(path)[0], "%Y-%m-%d") <= end_date
]

# -----------------------------
# 加载概念词并构建 Aho–Corasick 自动机
# -----------------------------
concepts_files = os.path.join(concept_folder, 'full_domain_concept.txt')
with open(concepts_files, 'r', encoding='utf-8') as file:
    full_concepts = [concept.strip().lower() for concept in file.readlines() if concept.strip()]

A = ahocorasick.Automaton()
for i, concept in enumerate(full_concepts):
    A.add_word(concept, i)
A.make_automaton()

print(f"✅ 加载 {len(full_concepts)} 个概念，自动机构建完成。")

# -----------------------------
# 断点续跑设置
# -----------------------------
if os.path.exists(progress_file):
    with open(progress_file, "r") as f:
        finished_ids = set(json.load(f))
else:
    finished_ids = set()

paper_starting_date = date(1990, 1, 1)
batch_size = 5  # 每批处理多少文件
batch_save_every = 50000  # 每隔多少条写入一次 gzip 文件

remaining_ids = [i for i in range(len(curr_run_file_paths)) if i not in finished_ids]

# -----------------------------
# 主循环
# -----------------------------
for curr_ID in remaining_ids[:batch_size]:
    formatted_ID = '{:03d}'.format(curr_ID)
    file_path = curr_run_file_paths[curr_ID]

    edge_file = os.path.join(edge_folder, f'edge_part_{formatted_ID}.gz')
    edge_file_log = os.path.join(edge_folder_log, f'edge_part_{formatted_ID}.txt')
    log_file_txt = os.path.join(log_folder, f'log_edge_part_{formatted_ID}.txt')
    log_file_txt_finish = os.path.join(log_folder, f'log_edge_part_{formatted_ID}_finish.txt')

    current_time = datetime.now()
    with open(log_file_txt, 'a') as log_file:
        log_file.write(f'Current time: {current_time}; Start File: {file_path}\n')

    edge_lists = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        lines = file.readlines()

        if not lines:
            with open(log_file_txt, 'a') as log_file:
                log_file.write("⚠️ File empty\n")
            continue

        for id_line, line in enumerate(lines):
            article_object = json.loads(line)
            get_date = datetime.strptime(article_object['publication_date'], "%Y-%m-%d").date()
            curr_paper_time = (get_date - paper_starting_date).days
            curr_all_citations = article_object.get('cited_by_count', 0)
            curr_citations_per_year = article_object.get('counts_by_year', [])
            curr_article = get_single_article_string(article_object)

            # 使用 Aho–Corasick 快速匹配概念
            concepts_for_single_paper = [idx for _, idx in A.iter(curr_article)]

            for ii in range(len(concepts_for_single_paper)):
                for jj in range(ii + 1, len(concepts_for_single_paper)):
                    edge_lists.append([
                        concepts_for_single_paper[ii],
                        concepts_for_single_paper[jj],
                        curr_paper_time,
                        curr_all_citations,
                        curr_citations_per_year
                    ])

            if len(edge_lists) >= batch_save_every:
                with gzip.open(edge_file, 'ab') as output_file:
                    pickle.dump(edge_lists, output_file)
                edge_lists = []  # 清空缓存

            if id_line % 5000 == 0:
                with open(log_file_txt, 'a') as log_file:
                    log_file.write(f"Processed {id_line}/{len(lines)} lines\n")

    # 保存剩余数据
    if edge_lists:
        with gzip.open(edge_file, 'ab') as output_file:
            pickle.dump(edge_lists, output_file)

    with open(edge_file_log, 'a') as log_file:
        log_file.write(f'\nedge_list={len(edge_lists)}')

    with open(log_file_txt_finish, 'a') as log_file:
        log_file.write(f'Finish File: {file_path}; Time: {datetime.now()}\n')

    # 更新进度
    finished_ids.add(curr_ID)
    with open(progress_file, "w") as f:
        json.dump(list(finished_ids), f)

print(f"🎉 所有批次完成（共 {len(finished_ids)} 个文件）")


✅ 加载 37960 个概念，自动机构建完成。
🎉 所有批次完成（共 5 个文件）


In [2]:
import glob
import gzip
import json
import os
import time
from datetime import datetime, date
import pickle
from functools import reduce
import random
import re

def get_single_article_string(article):
    """
    从文章对象中提取标题和摘要，并进行文本预处理
    """
    try:
        # 安全地获取标题，如果不存在则使用空字符串
        curr_title = article.get('title', '')
        if curr_title is None:
            curr_title = ''
            
        # 安全地获取摘要倒排索引
        abstract_inverted_index = article.get('abstract_inverted_index')
        
        if abstract_inverted_index is None:
            # 如果没有摘要，只使用标题
            curr_abstract = ""
        else:
            # Flatten the inverted index into a list of (position, word) tuples
            position_word_list = []
            for word, positions in abstract_inverted_index.items():
                if positions:  # 确保 positions 列表不为空
                    for position in positions:
                        position_word_list.append((position, word))
            
            # Sort the list by position and extract the words
            if position_word_list:
                sorted_abstract = sorted(position_word_list)
                curr_abstract = ' '.join(word for position, word in sorted_abstract)
            else:
                curr_abstract = ""

        # Replace strings according to the replace_pairs list
        replace_pairs = [
            ['\n',' '], ['-',' '], [' \" a','oa'], ['\" a','ae'], ['\"a','ae'],
            [' \" o','oe'], ['\" o','oe'], ['\"o','oe'], [' \" u','ue'],
            ['\" u','ue'], ['\"u','ue'], [' \' a','a'], [' \' e','e'],
            [' \' o','o'], ["\' ", ""], ["\'", ""], ['  ',' '], ['  ',' ']
        ]

        article_string = (curr_title + ' ' + curr_abstract).lower()
        article_string = reduce(lambda text, pair: text.replace(pair[0], pair[1]), replace_pairs, article_string)

        return article_string
        
    except Exception as e:
        # 如果处理过程中出现任何错误，返回空字符串并记录错误
        print(f"处理文章时出错: {e}")
        return ""

# Define a sorting key function to extract the date and part number from the path
def get_date_and_part_from_path(path):
    """
    从文件路径中提取日期和部分编号
    """
    try:
        date_folder = os.path.dirname(path)
        date_str = date_folder.split('=')[-1]

        file_name = os.path.basename(path)
        part_str = file_name.split('_')[-1].split('.')[0]

        return date_str, int(part_str)
    except Exception as e:
        print(f"解析文件路径时出错 {path}: {e}")
        return "1900-01-01", 0

def extract_id(filename):
    """
    从日志文件名中提取ID
    """
    try:
        match = re.search(r'log_edge_part_(\d+)_', filename)
        if match:
            return int(match.group(1))
        return None
    except Exception as e:
        print(f"提取ID时出错 {filename}: {e}")
        return None

def safe_json_loads(line, line_number):
    """
    安全地解析JSON行数据
    """
    try:
        return json.loads(line)
    except json.JSONDecodeError as e:
        print(f"第 {line_number} 行JSON解析错误: {e}")
        return None
    except Exception as e:
        print(f"第 {line_number} 行解析时出现未知错误: {e}")
        return None

def safe_date_parsing(date_str, line_number):
    """
    安全地解析日期
    """
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError as e:
        print(f"第 {line_number} 行日期解析错误 '{date_str}': {e}")
        return None
    except Exception as e:
        print(f"第 {line_number} 行日期解析时出现未知错误: {e}")
        return None

# 创建必要的目录
log_folder = 'logs_pair'
edge_folder = 'concept_pair'
edge_folder_log = 'concept_pair_log'

try:
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)
    
    if not os.path.exists(edge_folder):
        os.makedirs(edge_folder)
        
    if not os.path.exists(edge_folder_log):
        os.makedirs(edge_folder_log)

except FileExistsError:
    pass
except Exception as e:
    print(f"创建目录时出错: {e}")
    exit(1)

# 设置概念文件路径
data_folder = "data_concept_graph"
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
concept_folder = os.path.join(parent_dir, data_folder)

# 定义基础路径和文件模式
base_folder = 'D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works/'
date_pattern = 'updated_date=*'
file_pattern = 'part_*.gz'

print("=== 开始处理 ===")
print(f"基础路径: {base_folder}")
print(f"基础路径是否存在: {os.path.exists(base_folder)}")

# 查找所有匹配的文件
file_paths = glob.glob(f'{base_folder}/{date_pattern}/{file_pattern}')
print(f"找到的文件路径数量: {len(file_paths)}")

if len(file_paths) == 0:
    print("❌ 错误: 没有找到任何数据文件!")
    print("请检查以下可能的问题:")
    print("1. 基础路径是否正确")
    print("2. 文件命名模式是否匹配")
    print("3. 文件是否存在")
    exit(1)

# 按日期和部分编号排序文件
file_paths = sorted(file_paths, key=get_date_and_part_from_path)

# 定义时间范围
start_date = datetime.strptime("2022-12-20", "%Y-%m-%d")
end_date = datetime.strptime("2025-10-01", "%Y-%m-%d")
print(f"处理时间范围: {start_date} 到 {end_date}")

# 根据时间范围过滤文件
curr_run_file_paths = []
for path in file_paths:
    try:
        date_str, part_num = get_date_and_part_from_path(path)
        file_date = datetime.strptime(date_str, "%Y-%m-%d")
        if start_date <= file_date <= end_date:
            curr_run_file_paths.append(path)
    except Exception as e:
        print(f"处理文件路径时出错 {path}: {e}")

print(f"在时间范围内的文件数: {len(curr_run_file_paths)}")

if len(curr_run_file_paths) == 0:
    print("❌ 错误: 在指定时间范围内没有找到文件!")
    exit(1)

# 读取概念列表
concepts_files = os.path.join(concept_folder, 'full_domain_concept.txt')
print(f"概念文件路径: {concepts_files}")
print(f"概念文件是否存在: {os.path.exists(concepts_files)}")

if not os.path.exists(concepts_files):
    print("❌ 错误: 概念文件不存在!")
    exit(1)

try:
    with open(concepts_files, 'r', encoding='utf-8') as file:
        full_concepts = [concept.strip() for concept in file.readlines()]
    print(f"成功加载概念数量: {len(full_concepts)}")
except Exception as e:
    print(f"读取概念文件时出错: {e}")
    exit(1)

# 初始化变量
paper_starting_date = date(1990, 1, 1)
write_file = 0

# 随机延迟以避免冲突
rnd_time = random.random() * 60
time.sleep(rnd_time)

print(f"开始处理 {len(curr_run_file_paths)} 个文件...")

# 主处理循环
while write_file < len(curr_run_file_paths):
    
    curr_ID = random.randint(0, len(curr_run_file_paths) - 1)
    formatted_ID = '{:03d}'.format(curr_ID)

    edge_file = os.path.join(edge_folder, 'edge_part_' + formatted_ID + '.gz')
    edge_file_log = os.path.join(edge_folder_log, 'edge_part_' + formatted_ID + '.txt')
    log_file_txt = os.path.join(log_folder, 'log_edge_part_' + formatted_ID + '.txt')
    log_file_txt_finish = os.path.join(log_folder, 'log_edge_part_' + formatted_ID + '_finish.txt')
    log_file_txt_empty = os.path.join(log_folder, 'log_edge_part_' + formatted_ID + '_empty.txt')

    if not os.path.exists(log_file_txt):
        current_time = datetime.now()
        open(log_file_txt, 'a').close()
            
        file_path = curr_run_file_paths[curr_ID]
        print(f"开始处理文件 {write_file + 1}/{len(curr_run_file_paths)}: {file_path}")
        
        with open(log_file_txt, 'a', encoding='utf-8') as log_file:
            log_file.write(f'Current time: {current_time}; Number of files: {len(curr_run_file_paths)}; Number of concepts: {len(full_concepts)}\n\n')
            log_file.write(f'Start the File: {file_path}; Current time: {datetime.now()} \n\n')
            
        try:
            with gzip.open(file_path, 'rt', encoding='utf-8') as file:
                lines = file.readlines()

            if not lines:
                print(f'文件 {file_path} 为空')
                write_file += 1
                with open(log_file_txt_empty, 'a', encoding='utf-8') as log_file:
                    log_file.write(f'Current File: {file_path}; Paper: {len(lines)}; File is Empty!\n')
            else:
                edge_lists = []
                processed_papers = 0
                error_count = 0
                
                for id_line, line in enumerate(lines):
                    time_start_line = time.time()

                    # 安全地解析JSON
                    article_object = safe_json_loads(line, id_line + 1)
                    if article_object is None:
                        error_count += 1
                        continue

                    try:
                        # 安全地获取和处理文章数据
                        publication_date = article_object.get('publication_date')
                        if not publication_date:
                            error_count += 1
                            continue

                        get_date = safe_date_parsing(publication_date, id_line + 1)
                        if get_date is None:
                            error_count += 1
                            continue

                        curr_paper_time = (get_date - paper_starting_date).days
                        curr_all_citations = article_object.get('cited_by_count', 0)
                        curr_citations_per_year = article_object.get('counts_by_year', [])
                        curr_article = get_single_article_string(article_object)

                        # 检查文章是否包含任何概念
                        concepts_for_single_paper = []
                        for id_concept, concept in enumerate(full_concepts):
                            if concept and concept in curr_article:
                                concepts_for_single_paper.append(id_concept)

                        # 生成概念对
                        for ii in range(len(concepts_for_single_paper)):
                            for jj in range(ii + 1, len(concepts_for_single_paper)):
                                edge_lists.append([
                                    concepts_for_single_paper[ii],
                                    concepts_for_single_paper[jj],
                                    curr_paper_time,
                                    curr_all_citations,
                                    curr_citations_per_year
                                ])

                        processed_papers += 1

                    except Exception as e:
                        error_count += 1
                        print(f"处理第 {id_line + 1} 行文章数据时出错: {e}")
                        continue

                    # 记录进度
                    if id_line % 10000 == 0 and id_line > 0:
                        progress = (id_line + 1) / len(lines) * 100
                        with open(log_file_txt, 'a', encoding='utf-8') as log_file:
                            log_file.write(f'Current File: {file_path}; Paper: {len(lines)}; Processed: {progress:.2f}%; time: {time.time()-time_start_line:.2f}s\n')

                # 保存边列表
                try:
                    with gzip.open(edge_file, 'wb') as output_file:
                        pickle.dump(edge_lists, output_file)
                    write_file += 1
                    
                    with open(edge_file_log, 'a', encoding='utf-8') as log_file:
                        log_file.write(f'\nedge_list={len(edge_lists)}')
                    
                    with open(log_file_txt, 'a', encoding='utf-8') as log_file:
                        log_file.write(f'\n\nFinish Time: {datetime.now()}; Current File: {file_path}; Processed: {write_file}/{len(curr_run_file_paths)}, i.e., {write_file/len(curr_run_file_paths)*100:.2f}% \n')
                        log_file.write(f'\nedge_list: {len(edge_lists)}\n')
                        log_file.write(f'成功处理文章: {processed_papers}\n')
                        log_file.write(f'处理错误: {error_count}\n')
                    
                    with open(log_file_txt_finish, 'a', encoding='utf-8') as log_file:
                        log_file.write(f'\n\nFinish Time: {datetime.now()}; Current File: {file_path} \n')
                    
                    print(f"完成文件 {write_file}/{len(curr_run_file_paths)}: 生成 {len(edge_lists)} 条边, 成功处理 {processed_papers} 篇文章, 错误 {error_count} 个")
                    
                except Exception as e:
                    print(f"保存边列表时出错: {e}")
                    write_file += 1  # 仍然计数，避免无限循环

                rnd_time = random.random() * 5
                time.sleep(rnd_time)
                
        except Exception as e:
            print(f"读取文件 {file_path} 时出错: {e}")
            write_file += 1  # 仍然计数，避免无限循环
                
    else:
        # 检查是否所有文件都已处理完成
        finish_pattern = os.path.join(log_folder, 'log_edge_part_*_finish.txt')
        empty_pattern = os.path.join(log_folder, 'log_edge_part_*_empty.txt')
        
        try:
            finished_files = [f for f in glob.glob(finish_pattern) if extract_id(f) is not None and extract_id(f) < len(curr_run_file_paths)]
            empty_files = [f for f in glob.glob(empty_pattern) if extract_id(f) is not None and extract_id(f) < len(curr_run_file_paths)]

            total_files = len(finished_files) + len(empty_files)

            if total_files >= len(curr_run_file_paths):
                print(f"{datetime.now()}: 所有文件处理完成!")
                break
        except Exception as e:
            print(f"检查完成状态时出错: {e}")

# 写入完成标记
try:
    with open("job_finish.txt", 'a', encoding='utf-8') as f:
        f.write(f'\n完成所有处理: {datetime.now()}\n')
    print("处理完成!")
except Exception as e:
    print(f"写入完成标记时出错: {e}")

=== 开始处理 ===
基础路径: D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works/
基础路径是否存在: True
找到的文件路径数量: 873
处理时间范围: 2022-12-20 00:00:00 到 2025-10-01 00:00:00
在时间范围内的文件数: 873
概念文件路径: d:\Impact4Cast-main\Impact4Cast-main\Impact4Cast-main\data_concept_graph\full_domain_concept.txt
概念文件是否存在: True
成功加载概念数量: 37960
开始处理 873 个文件...
开始处理文件 1/873: D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works\updated_date=2025-04-22\part_001.gz
完成文件 1/873: 生成 720950 条边, 成功处理 281988 篇文章, 错误 0 个
开始处理文件 2/873: D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works\updated_date=2025-01-20\part_010.gz
完成文件 2/873: 生成 122332 条边, 成功处理 802025 篇文章, 错误 0 个
开始处理文件 3/873: D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works\updated_date=2024-12-08\part_000.gz
完成文件 3/873: 生成 408554 条边, 成功处理 756100 篇文章, 错误 0 个
开始处理文件 4/873: D:/Imp

KeyboardInterrupt: 

In [14]:
# 推荐在 conda / venv 下
! pip install -U sentence-transformers faiss-gpu torch tqdm
# 如果 faiss-gpu 安装出问题，根据你的cuda版本选择合适的faiss包（比如 conda install -c pytorch faiss-gpu）


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# run_ac_gpu_pipeline.py
"""
AC 自动机 + 多进程 + 流式读取 + 分段保存 + 断点续跑
适用场景：大量 gz 压缩 JSON 文件，每行为一篇文章（OpenAlex 格式似乎类似）
输出：concept_pair/edge_part_{part}.pkl.gz （append 模式写入多个 pickle-chunks）
每个源文件对应的进度文件 logs_pair/progress/progress_{part}.txt
"""

import os
import gzip
import json
import re
import time
import pickle
from datetime import datetime, date
from functools import partial
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# 尝试导入可选 GPU 库（用于可选的后处理加速）
try:
    import cupy as cp
    CUPY_AVAILABLE = True
except Exception:
    CUPY_AVAILABLE = False

# AC 自动机库
import ahocorasick

# ---------------------------
# 配置（按需修改）
# ---------------------------
BASE_FOLDER = r'D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works'
# BASE_FOLDER 目录下包含 updated_date=YYYY-MM-DD/.../part_xxx.gz
DATE_PATTERN = 'updated_date='
FILE_PATTERN_PREFIX = 'part_'
FILE_PATTERN_SUFFIX = '.gz'

# 概念文件路径（与你之前的组织一致）
DATA_FOLDER = "data_concept_graph"
CONCEPT_FILE = os.path.join(os.path.dirname(os.getcwd()), DATA_FOLDER, 'full_domain_concept.txt')

EDGE_FOLDER = 'concept_pair'
LOG_FOLDER = 'logs_pair'
PROGRESS_FOLDER = os.path.join(LOG_FOLDER, 'progress')

# 保存/恢复策略
EDGE_SAVE_BATCH_LINES = 50000     # 每处理多少行写一次进度（保存边）
FLUSH_EDGES_COUNT = 200_000      # 边缓存达到多少就写磁盘（避免太大内存）
WRITE_CHUNK_SIZE = 50_000        # 每多少条边写一次（更细粒度）
WORKERS = max(1, cpu_count() - 1)

# 过滤与时间范围（保持你原逻辑）
START_DATE = datetime.strptime("2022-12-20", "%Y-%m-%d")
END_DATE = datetime.strptime("2025-10-01", "%Y-%m-%d")

# 日志级别（是否在处理时打印过多信息）
VERBOSE = True

# ---------------------------
# 全局（由初始化函数在 worker 进程中设置）
# ---------------------------
AUTOMATON = None        # ahocorasick.Automaton 对象（包含 concept -> id）
CONCEPT_LIST = None     # index -> concept 字符串
PAPER_STARTING_DATE = date(1990, 1, 1)

# ---------------------------
# 辅助函数（与原逻辑保持一致）
# ---------------------------
def get_single_article_string(article):
    """
    从文章对象中提取标题和摘要，并进行文本预处理
    """
    try:
        curr_title = article.get('title', '') or ''
        abstract_inverted_index = article.get('abstract_inverted_index')
        if abstract_inverted_index:
            position_word_list = []
            for word, positions in abstract_inverted_index.items():
                if positions:
                    for position in positions:
                        position_word_list.append((position, word))
            if position_word_list:
                sorted_abstract = sorted(position_word_list)
                curr_abstract = ' '.join(word for position, word in sorted_abstract)
            else:
                curr_abstract = ""
        else:
            curr_abstract = ""

        replace_pairs = [
            ['\n',' '], ['-',' '], [' \" a','oa'], ['\" a','ae'], ['\"a','ae'],
            [' \" o','oe'], ['\" o','oe'], ['\"o','oe'], [' \" u','ue'],
            ['\" u','ue'], ['\"u','ue'], [' \' a','a'], [' \' e','e'],
            [' \' o','o'], ["\' ", ""], ["\'", ""], ['  ',' '], ['  ',' ']
        ]
        article_string = (curr_title + ' ' + curr_abstract).lower()
        for a,b in replace_pairs:
            article_string = article_string.replace(a,b)
        return article_string
    except Exception as e:
        # 若处理失败直接返回空字符串
        if VERBOSE:
            print("get_single_article_string 出错:", e)
        return ""

def safe_json_loads(line, line_number=None):
    try:
        return json.loads(line)
    except Exception as e:
        # 解析失败返回 None（避免频繁打印）
        return None

def safe_date_parsing(date_str):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except Exception:
        return None

def get_date_and_part_from_path(path):
    """
    从路径推断日期和 part 编号（用于排序）
    """
    try:
        # 假设路径类似 .../updated_date=2023-01-01/part_0001.gz
        date_folder = os.path.dirname(path)
        date_str = date_folder.split('=')[-1]
        file_name = os.path.basename(path)
        part_str = file_name.split('_')[-1].split('.')[0]
        return date_str, int(part_str)
    except Exception:
        return "1900-01-01", 0

def extract_id_from_logname(path):
    """
    从 log 文件名中提取 part id（用于旧脚本的兼容）
    """
    try:
        m = re.search(r'part_(\d+)', path)
        return int(m.group(1)) if m else None
    except Exception:
        return None

# ---------------------------
# 初始化 worker（在每个进程中构建 automaton）
# ---------------------------
def init_worker(concept_file_path):
    """
    每个 worker 在启动时执行一次，构建本进程的 AC 自动机（节省 IPC/序列化问题）
    """
    global AUTOMATON, CONCEPT_LIST
    # 读取概念文件
    if not os.path.exists(concept_file_path):
        raise FileNotFoundError(f"概念文件不存在: {concept_file_path}")

    # 逐行读取概念，并把较短/空行过滤掉
    with open(concept_file_path, 'r', encoding='utf-8') as cf:
        concepts = [line.strip().lower() for line in cf if line.strip()]
    CONCEPT_LIST = concepts

    # 构建 AC 自动机，value 存为概念 id（索引）
    A = ahocorasick.Automaton()
    for idx, concept in enumerate(concepts):
        # 跳过过短或过长的概念（根据需要）
        if not concept:
            continue
        A.add_word(concept, idx)
    A.make_automaton()
    AUTOMATON = A
    if VERBOSE:
        print(f"[worker init] Loaded {len(concepts)} concepts into AC automaton")

# ---------------------------
# 处理单个文件的主函数（在 worker 中执行）
# ---------------------------
def process_file_worker(file_path):
    """
    在 worker 里运行：
    - 流式解压文件（gzip）逐行读取
    - 每行解析 JSON，构造 article_string
    - 用 AC 自动机匹配概念（返回概念 id 列表）
    - 生成概念对并累积到 edge_lists
    - 定期写入 edge 输出文件，并记录 progress
    """
    global AUTOMATON, CONCEPT_LIST, PAPER_STARTING_DATE
    fname = os.path.basename(file_path)
    # 提取 part id 作为输出文件编号
    m = re.search(r'part_(\d+)\.gz', fname)
    part_id = m.group(1) if m else fname.replace('.gz','')

    edge_out_file = os.path.join(EDGE_FOLDER, f'edge_part_{part_id}.pkl.gz')
    progress_file = os.path.join(PROGRESS_FOLDER, f'progress_{part_id}.txt')
    log_file = os.path.join(LOG_FOLDER, f'log_edge_part_{part_id}.txt')
    log_file_finish = os.path.join(LOG_FOLDER, f'log_edge_part_{part_id}_finish.txt')
    log_file_empty = os.path.join(LOG_FOLDER, f'log_edge_part_{part_id}_empty.txt')

    # 读取上次处理进度（行号）
    start_line = 0
    if os.path.exists(progress_file):
        try:
            with open(progress_file, 'r', encoding='utf-8') as pf:
                raw = pf.read().strip()
                if raw:
                    start_line = int(raw)
            if VERBOSE:
                print(f"[resume] {fname} 从行 {start_line} 开始恢复")
        except Exception:
            start_line = 0

    edges_accum = []
    processed_papers = 0
    error_count = 0
    last_saved_line = start_line
    any_line_seen = False

    try:
        with gzip.open(file_path, 'rt', encoding='utf-8', errors='ignore') as fh:
            for line_id, line in enumerate(fh):
                any_line_seen = True
                if line_id < start_line:
                    continue

                article = safe_json_loads(line, line_id + 1)
                if article is None:
                    error_count += 1
                    continue

                pub_date = article.get('publication_date')
                if not pub_date:
                    error_count += 1
                    continue
                parsed_date = safe_date_parsing(pub_date)
                if parsed_date is None:
                    error_count += 1
                    continue
                # 时间过滤
                if not (START_DATE.date() <= parsed_date <= END_DATE.date()):
                    # 如果不在时间范围则跳过
                    continue

                curr_paper_time = (parsed_date - PAPER_STARTING_DATE).days
                curr_all_citations = article.get('cited_by_count', 0)
                curr_citations_per_year = article.get('counts_by_year', [])

                article_string = get_single_article_string(article)
                if not article_string:
                    # 没有有用文本
                    continue

                # AC 自动机匹配（返回 pattern 的 value，即 concept id）
                matched_ids = set()
                for end_index, value in AUTOMATON.iter(article_string):
                    # value 本身是 concept id（我们在 init_worker 时设置）
                    matched_ids.add(value)

                if matched_ids:
                    # 转成排序的 list（保证顺序稳定）
                    matched_sorted = sorted(matched_ids)
                    # 生成概念对（组合）
                    L = len(matched_sorted)
                    for i in range(L):
                        for j in range(i+1, L):
                            edges_accum.append([
                                matched_sorted[i],
                                matched_sorted[j],
                                curr_paper_time,
                                curr_all_citations,
                                curr_citations_per_year
                            ])

                processed_papers += 1

                # 定期写进度与中间结果（按行数）
                if (line_id - last_saved_line) >= EDGE_SAVE_BATCH_LINES:
                    # 写一次边到磁盘
                    if edges_accum:
                        _append_edges_to_disk(edge_out_file, edges_accum)
                        edges_accum.clear()
                    # 写进度
                    with open(progress_file, 'w', encoding='utf-8') as pf:
                        pf.write(str(line_id))
                    last_saved_line = line_id
                    if VERBOSE:
                        print(f"[{fname}] 行 {line_id} - 保存中间结果与进度")

                # 若边积累过大，也写磁盘（避免 OOM）
                if len(edges_accum) >= FLUSH_EDGES_COUNT:
                    _append_edges_to_disk(edge_out_file, edges_accum)
                    edges_accum.clear()

        # 处理完文件：保存剩余边
        if not any_line_seen:
            # 文件为空
            with open(log_file_empty, 'a', encoding='utf-8') as lf:
                lf.write(f'Empty file: {file_path}; time: {datetime.now()}\n')
            return {'file': fname, 'status': 'empty'}

        if edges_accum:
            _append_edges_to_disk(edge_out_file, edges_accum)
            edges_accum.clear()

        # 写最终进度（标注为完成行号）
        # 如果想标注 _finish 文件也可
        with open(progress_file, 'w', encoding='utf-8') as pf:
            pf.write(str(line_id))

        with open(log_file, 'a', encoding='utf-8') as lf:
            lf.write(f'\nFinish Time: {datetime.now()}; File: {file_path}; processed_papers: {processed_papers}; errors: {error_count}\n')

        with open(log_file_finish, 'a', encoding='utf-8') as lf:
            lf.write(f'Finish Time: {datetime.now()}; File: {file_path}\n')

        if VERBOSE:
            print(f"[DONE] {fname}: processed {processed_papers}, errors {error_count}")

        return {'file': fname, 'status': 'done', 'processed': processed_papers, 'errors': error_count}

    except Exception as e:
        # 出错时仍记录并返回
        with open(log_file, 'a', encoding='utf-8') as lf:
            lf.write(f'Error processing {file_path} at {datetime.now()}: {e}\n')
        if VERBOSE:
            print(f"[ERROR] 处理文件 {file_path} 出错: {e}")
        return {'file': fname, 'status': 'error', 'error': str(e)}

def _append_edges_to_disk(edge_out_file, edges_list):
    """
    将 edges_list (list of edges) 追加写入 edge_out_file (gzip + pickle)
    这样保存的是多个 pickle chunk，后面合并时需要迭代读取每个 chunk。
    """
    try:
        # 以二进制 append 模式写入 gzip
        with gzip.open(edge_out_file, 'ab') as gf:
            # 分批写，避免一次 pickle 太大（可按 WRITE_CHUNK_SIZE 分块）
            start = 0
            n = len(edges_list)
            while start < n:
                end = min(start + WRITE_CHUNK_SIZE, n)
                chunk = edges_list[start:end]
                pickle.dump(chunk, gf, protocol=pickle.HIGHEST_PROTOCOL)
                start = end
    except Exception as e:
        # 若写失败，打印日志（但不要 raise，以保证继续处理其他文件）
        print("写 edge 文件时出错:", e)

# ---------------------------
# 主执行入口（负责收集待处理文件与启动进程池）
# ---------------------------
def collect_file_paths(base_folder):
    """
    根据 BASE_FOLDER 查找所有包含 updated_date= 的子文件夹下的 part_*.gz 文件
    并按日期+part 编号排序，返回路径列表
    """
    file_paths = []
    for root, dirs, files in os.walk(base_folder):
        if DATE_PATTERN in root:
            for fn in files:
                if fn.startswith(FILE_PATTERN_PREFIX) and fn.endswith(FILE_PATTERN_SUFFIX):
                    file_paths.append(os.path.join(root, fn))
    # 排序
    file_paths = sorted(file_paths, key=get_date_and_part_from_path)
    return file_paths

def main():
    # 确保必要目录存在
    os.makedirs(EDGE_FOLDER, exist_ok=True)
    os.makedirs(LOG_FOLDER, exist_ok=True)
    os.makedirs(PROGRESS_FOLDER, exist_ok=True)

    # 检查概念文件
    if not os.path.exists(CONCEPT_FILE):
        raise FileNotFoundError(f"概念文件不存在: {CONCEPT_FILE}")

    # 收集文件列表并根据时间范围过滤（与原逻辑一致）
    all_files = collect_file_paths(BASE_FOLDER)
    if not all_files:
        print("未找到任何数据文件，请检查 BASE_FOLDER 路径与模式。")
        return

    # 进一步时间范围过滤（如果你想）
    curr_run_files = []
    for p in all_files:
        try:
            date_str, part_num = get_date_and_part_from_path(p)
            file_date = datetime.strptime(date_str, "%Y-%m-%d")
            if START_DATE <= file_date <= END_DATE:
                curr_run_files.append(p)
        except Exception:
            # 忽略无法解析的
            continue

    if not curr_run_files:
        print("在给定时间范围内未找到文件。")
        return

    print(f"找到 {len(curr_run_files)} 个待处理文件，启动 {WORKERS} 个 worker ...")

    # 使用 Pool 并用 initializer 在 worker 中加载 automaton（避免主进程序列化大型 automaton）
    pool = Pool(processes=WORKERS, initializer=init_worker, initargs=(CONCEPT_FILE,))

    try:
        # 使用 imap_unordered 以便尽快得到完成反馈
        for res in tqdm(pool.imap_unordered(process_file_worker, curr_run_files), total=len(curr_run_files)):
            # res 是 dict 状态，可根据需要记录或打印
            if VERBOSE:
                print("Task result:", res)
    finally:
        pool.close()
        pool.join()

    print("所有任务提交完毕，处理结束。")

# ---------------------------
# 合并小块 edge 文件的示例（供你后续运行）
# ---------------------------
def merge_all_edges(output_file='all_edges_merged.pkl.gz', input_folder=EDGE_FOLDER):
    """
    合并 input_folder 下所有 edge_part_*.pkl.gz 文件为一个 gzip pickle 文件（列表）
    注意：若数据非常大，不建议一次性合并到内存中，而应按下游任务逐 chunk 读取。
    """
    files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.startswith('edge_part_') and f.endswith('.pkl.gz')]
    all_edges = []
    for fpath in files:
        with gzip.open(fpath, 'rb') as gf:
            while True:
                try:
                    chunk = pickle.load(gf)
                    all_edges.extend(chunk)
                except EOFError:
                    break
    # 写到一个大文件
    with gzip.open(output_file, 'wb') as outf:
        pickle.dump(all_edges, outf)
    print("合并完成，total edges:", len(all_edges))

# ---------------------------
# 如果需要，用 GPU(CuPy)对边进行去重/统计的示例（可选）
# ---------------------------
def deduplicate_edges_gpu(edge_list):
    """
    使用 cupy 在 GPU 上对边进行去重（edge_list: list of [i,j,...])
    返回 numpy 形式的唯一边数组
    仅当 CUPY_AVAILABLE 时可用
    """
    if not CUPY_AVAILABLE:
        raise RuntimeError("CuPy 未安装或不可用")
    arr = cp.array(edge_list)
    # 对二维数组按行去重
    uniq = cp.unique(arr, axis=0)
    return cp.asnumpy(uniq)

# ---------------------------
# 脚本入口
# ---------------------------
if __name__ == '__main__':
    main()


找到 873 个待处理文件，启动 15 个 worker ...


  0%|          | 0/873 [00:00<?, ?it/s]

In [ ]:
import glob
import gzip
import json
import os
import time
import pickle
import random
import re
from datetime import datetime, date
from functools import reduce

# 建议安装：pip install pyahocorasick tqdm
try:
    import ahocorasick
    AHO_AVAILABLE = True
except Exception:
    AHO_AVAILABLE = False

# 可选 GPU 去重（若未安装则会跳过）
try:
    import cupy as cp
    CUPY_AVAILABLE = True
except Exception:
    CUPY_AVAILABLE = False

# ----------------------
# 你的原始函数（几乎未改动）
# ----------------------
def get_single_article_string(article):
    """
    从文章对象中提取标题和摘要，并进行文本预处理
    """
    try:
        curr_title = article.get('title', '')
        if curr_title is None:
            curr_title = ''
            
        abstract_inverted_index = article.get('abstract_inverted_index')
        
        if abstract_inverted_index is None:
            curr_abstract = ""
        else:
            position_word_list = []
            for word, positions in abstract_inverted_index.items():
                if positions:
                    for position in positions:
                        position_word_list.append((position, word))
            if position_word_list:
                sorted_abstract = sorted(position_word_list)
                curr_abstract = ' '.join(word for position, word in sorted_abstract)
            else:
                curr_abstract = ""

        replace_pairs = [
            ['\n',' '], ['-',' '], [' \" a','oa'], ['\" a','ae'], ['\"a','ae'],
            [' \" o','oe'], ['\" o','oe'], ['\"o','oe'], [' \" u','ue'],
            ['\" u','ue'], ['\"u','ue'], [' \' a','a'], [' \' e','e'],
            [' \' o','o'], ["\' ", ""], ["\'", ""], ['  ',' '], ['  ',' ']
        ]

        article_string = (curr_title + ' ' + curr_abstract).lower()
        article_string = reduce(lambda text, pair: text.replace(pair[0], pair[1]), replace_pairs, article_string)

        return article_string
        
    except Exception as e:
        print(f"处理文章时出错: {e}")
        return ""

# 原有辅助函数（保留）2023
def get_date_and_part_from_path(path):
    try:
        date_folder = os.path.dirname(path)
        date_str = date_folder.split('=')[-1]
        file_name = os.path.basename(path)
        part_str = file_name.split('_')[-1].split('.')[0]
        return date_str, int(part_str)
    except Exception as e:
        print(f"解析文件路径时出错 {path}: {e}")
        return "1900-01-01", 0

def extract_id(filename):
    try:
        match = re.search(r'log_edge_part_(\d+)_', filename)
        if match:
            return int(match.group(1))
        return None
    except Exception as e:
        print(f"提取ID时出错 {filename}: {e}")
        return None

def safe_json_loads(line, line_number):
    try:
        return json.loads(line)
    except json.JSONDecodeError as e:
        print(f"第 {line_number} 行JSON解析错误: {e}")
        return None
    except Exception as e:
        print(f"第 {line_number} 行解析时出现未知错误: {e}")
        return None

def safe_date_parsing(date_str, line_number):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError as e:
        print(f"第 {line_number} 行日期解析错误 '{date_str}': {e}")
        return None
    except Exception as e:
        print(f"第 {line_number} 行日期解析时出现未知错误: {e}")
        return None

# ----------------------
# 目录与路径（保持你给出的一致）
# ----------------------
log_folder = 'logs_pair'
edge_folder = 'concept_pair'
edge_folder_log = 'concept_pair_log'

try:
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)
    if not os.path.exists(edge_folder):
        os.makedirs(edge_folder)
    if not os.path.exists(edge_folder_log):
        os.makedirs(edge_folder_log)
except FileExistsError:
    pass
except Exception as e:
    print(f"创建目录时出错: {e}")
    exit(1)

# 你明确要求使用的概念文件路径（保持原样）
data_folder = "data_concept_graph"
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
concept_folder = os.path.join(parent_dir, data_folder)

# ----------------------
# 数据源设置（保持原样）
# ----------------------
base_folder = 'D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works/'
date_pattern = 'updated_date=*'
file_pattern = 'part_*.gz'

print("=== 开始处理 ===")
print(f"基础路径: {base_folder}")
print(f"基础路径是否存在: {os.path.exists(base_folder)}")

file_paths = glob.glob(f'{base_folder}/{date_pattern}/{file_pattern}')
print(f"找到的文件路径数量: {len(file_paths)}")

if len(file_paths) == 0:
    print("❌ 错误: 没有找到任何数据文件!")
    exit(1)

file_paths = sorted(file_paths, key=get_date_and_part_from_path)

start_date = datetime.strptime("2022-12-20", "%Y-%m-%d")
end_date = datetime.strptime("2025-10-01", "%Y-%m-%d")
print(f"处理时间范围: {start_date} 到 {end_date}")

curr_run_file_paths = []
for path in file_paths:
    try:
        date_str, part_num = get_date_and_part_from_path(path)
        file_date = datetime.strptime(date_str, "%Y-%m-%d")
        if start_date <= file_date <= end_date:
            curr_run_file_paths.append(path)
    except Exception as e:
        print(f"处理文件路径时出错 {path}: {e}")

print(f"在时间范围内的文件数: {len(curr_run_file_paths)}")

if len(curr_run_file_paths) == 0:
    print("❌ 错误: 在指定时间范围内没有找到文件!")
    exit(1)

# ----------------------
# 加载概念列表（使用你之前定义的 concept_folder）
# ----------------------
concepts_files = os.path.join(concept_folder, 'full_domain_concept.txt')
print(f"概念文件路径: {concepts_files}")
print(f"概念文件是否存在: {os.path.exists(concepts_files)}")

if not os.path.exists(concepts_files):
    print("❌ 错误: 概念文件不存在!")
    exit(1)

try:
    with open(concepts_files, 'r', encoding='utf-8') as file:
        # 小写化概念以和文章文本的小写化保持一致
        full_concepts = [concept.strip().lower() for concept in file.readlines() if concept.strip()]
    print(f"成功加载概念数量: {len(full_concepts)}")
except Exception as e:
    print(f"读取概念文件时出错: {e}")
    exit(1)

# ----------------------
# 构建 AC 自动机（一次）
# ----------------------
if not AHO_AVAILABLE:
    print("错误：未能导入 pyahocorasick，请先 pip install pyahocorasick")
    exit(1)

print("正在构建 Aho-Corasick 自动机（可能需要一点时间）...")
A = ahocorasick.Automaton()
for idx, concept in enumerate(full_concepts):
    # 跳过空串
    if not concept:
        continue
    # 添加概念：value 使用概念 id（整数）
    try:
        A.add_word(concept, idx)
    except Exception:
        # 某些非常长/异常模式可能导致错误，跳过它们
        continue
A.make_automaton()
print("AC 自动机构建完成")

# ----------------------
# 初始化变量 & 随机延迟（保留原逻辑）
# ----------------------
paper_starting_date = date(1990, 1, 1)
write_file = 0
rnd_time = random.random() * 60
time.sleep(rnd_time)
print(f"开始处理 {len(curr_run_file_paths)} 个文件...")

# 保存与恢复参数
EDGE_SAVE_BATCH_LINES = 50000  # 每处理多少源行就把当前边缓存写盘并更新进度
WRITE_CHUNK_SIZE = 50000       # 将边分块写入文件（避免一次 pickle 太大）
USE_GPU_DEDUP = CUPY_AVAILABLE # 若 cupy 可用则启用 GPU 去重，否则跳过

# ----------------------
# 主处理循环（基本保留原流程，但用流式读取并用 AC 匹配替代逐概念查找）
# ----------------------
while write_file < len(curr_run_file_paths):
    
    curr_ID = random.randint(0, len(curr_run_file_paths) - 1)
    formatted_ID = '{:03d}'.format(curr_ID)

    edge_file = os.path.join(edge_folder, 'edge_part_' + formatted_ID + '.gz')
    edge_file_log = os.path.join(edge_folder_log, 'edge_part_' + formatted_ID + '.txt')
    log_file_txt = os.path.join(log_folder, 'log_edge_part_' + formatted_ID + '.txt')
    log_file_txt_finish = os.path.join(log_folder, 'log_edge_part_' + formatted_ID + '_finish.txt')
    log_file_txt_empty = os.path.join(log_folder, 'log_edge_part_' + formatted_ID + '_empty.txt')

    if not os.path.exists(log_file_txt):
        current_time = datetime.now()
        open(log_file_txt, 'a').close()
            
        file_path = curr_run_file_paths[curr_ID]
        print(f"开始处理文件 {write_file + 1}/{len(curr_run_file_paths)}: {file_path}")
        
        with open(log_file_txt, 'a', encoding='utf-8') as log_file:
            log_file.write(f'Current time: {current_time}; Number of files: {len(curr_run_file_paths)}; Number of concepts: {len(full_concepts)}\n\n')
            log_file.write(f'Start the File: {file_path}; Current time: {datetime.now()} \n\n')
            
        try:
            # 流式读取，避免一次性 load 全文件
            with gzip.open(file_path, 'rt', encoding='utf-8', errors='ignore') as fh:
                # 我们不使用 readlines()，而逐行处理
                edge_lists = []
                processed_papers = 0
                error_count = 0
                last_saved_line = 0
                any_line_seen = False

                for id_line, line in enumerate(fh):
                    any_line_seen = True
                    time_start_line = time.time()

                    # 安全地解析JSON
                    article_object = safe_json_loads(line, id_line + 1)
                    if article_object is None:
                        error_count += 1
                        continue

                    try:
                        publication_date = article_object.get('publication_date')
                        if not publication_date:
                            error_count += 1
                            continue

                        get_date = safe_date_parsing(publication_date, id_line + 1)
                        if get_date is None:
                            error_count += 1
                            continue

                        curr_paper_time = (get_date - paper_starting_date).days
                        curr_all_citations = article_object.get('cited_by_count', 0)
                        curr_citations_per_year = article_object.get('counts_by_year', [])
                        curr_article = get_single_article_string(article_object)

                        # 使用 AC 自动机匹配概念（返回 concept id）
                        matched_ids = set()
                        if curr_article:
                            for end_idx, concept_id in A.iter(curr_article):
                                matched_ids.add(concept_id)

                        # 生成概念对（保持原来用概念 id 的格式）
                        matched_list = sorted(matched_ids)
                        for ii in range(len(matched_list)):
                            for jj in range(ii + 1, len(matched_list)):
                                edge_lists.append([
                                    matched_list[ii],
                                    matched_list[jj],
                                    curr_paper_time,
                                    curr_all_citations,
                                    curr_citations_per_year
                                ])

                        processed_papers += 1

                    except Exception as e:
                        error_count += 1
                        print(f"处理第 {id_line + 1} 行文章数据时出错: {e}")
                        continue

                    # 每隔一定行数记录进度并将边分段写入磁盘（避免内存过大）
                    if id_line % EDGE_SAVE_BATCH_LINES == 0 and id_line > 0:
                        # 按块写入 edge_file（gzip + pickle 的 append 模式）
                        try:
                            # 追加写入：将 edge_lists 分块写入避免单个 pickle 过大
                            with gzip.open(edge_file, 'ab') as output_file:
                                start_idx = 0
                                while start_idx < len(edge_lists):
                                    end_idx = min(start_idx + WRITE_CHUNK_SIZE, len(edge_lists))
                                    pickle.dump(edge_lists[start_idx:end_idx], output_file, protocol=pickle.HIGHEST_PROTOCOL)
                                    start_idx = end_idx
                            # 清空已写入的缓存
                            edge_lists.clear()
                            # 写进度日志
                            with open(log_file_txt, 'a', encoding='utf-8') as log_file:
                                log_file.write(f'Current File: {file_path}; Line: {id_line}; time: {time.time()-time_start_line:.2f}s\n')
                            # 写 empty/progress 标记文件（用于断点恢复）
                            with open(os.path.join(log_folder, f'progress_part_{formatted_ID}.txt'), 'w', encoding='utf-8') as pf:
                                pf.write(str(id_line))
                        except Exception as e:
                            print(f"写入中间 edge_file 出错: {e}")

                # 文件读取结束后的后处理
                if not any_line_seen:
                    print(f'文件 {file_path} 为空')
                    write_file += 1
                    with open(log_file_txt_empty, 'a', encoding='utf-8') as log_file:
                        log_file.write(f'Current File: {file_path}; Paper: 0; File is Empty!\n')
                else:
                    # 写剩余 edge_lists 到磁盘
                    try:
                        if edge_lists:
                            with gzip.open(edge_file, 'ab') as output_file:
                                start_idx = 0
                                while start_idx < len(edge_lists):
                                    end_idx = min(start_idx + WRITE_CHUNK_SIZE, len(edge_lists))
                                    pickle.dump(edge_lists[start_idx:end_idx], output_file, protocol=pickle.HIGHEST_PROTOCOL)
                                    start_idx = end_idx
                            edge_lists.clear()
                        # 成功写入
                        write_file += 1
                        
                        # 写 edge_file_log
                        with open(edge_file_log, 'a', encoding='utf-8') as log_file:
                            log_file.write(f'\nedge_list written for file: {file_path}\n')
                        
                        # 写主日志文件
                        with open(log_file_txt, 'a', encoding='utf-8') as log_file:
                            log_file.write(f'\n\nFinish Time: {datetime.now()}; Current File: {file_path}; Processed: {write_file}/{len(curr_run_file_paths)}, i.e., {write_file/len(curr_run_file_paths)*100:.2f}% \n')
                            log_file.write(f'\nedge_file saved (maybe multiple chunks)\n')
                            log_file.write(f'成功处理文章: {processed_papers}\n')
                            log_file.write(f'处理错误: {error_count}\n')
                        
                        # 写 finish 标记
                        with open(log_file_txt_finish, 'a', encoding='utf-8') as log_file:
                            log_file.write(f'\n\nFinish Time: {datetime.now()}; Current File: {file_path} \n')
                        
                        print(f"完成文件 {write_file}/{len(curr_run_file_paths)}: 生成边（写入到 {edge_file}），成功处理 {processed_papers} 篇文章, 错误 {error_count} 个")
                        
                        # 可选：在每个文件完成后进行 GPU 去重（如果可用）
                        if USE_GPU_DEDUP:
                            try:
                                print("尝试使用 GPU 去重该文件边（若数据量大会快很多）...")
                                # 读取所有 chunk，合并，使用 cupy 去重后覆盖保存（谨慎：要能装入 GPU 内存）
                                all_chunks = []
                                with gzip.open(edge_file, 'rb') as gf:
                                    while True:
                                        try:
                                            chunk = pickle.load(gf)
                                            all_chunks.extend(chunk)
                                        except EOFError:
                                            break
                                if all_chunks:
                                    arr = cp.array(all_chunks)  # shape (N, 5) 其中前两列为 concept ids
                                    # 只对前两列做去重（完整行去重）
                                    uniq = cp.unique(arr, axis=0)
                                    # 覆盖保存（写入 gzip）
                                    with gzip.open(edge_file, 'wb') as gf:
                                        # 将 cupy 数组转为列表再 dump（避免 pickle 直接存 cupy）
                                        uniq_list = cp.asnumpy(uniq).tolist()
                                        start_idx = 0
                                        while start_idx < len(uniq_list):
                                            end_idx = min(start_idx + WRITE_CHUNK_SIZE, len(uniq_list))
                                            pickle.dump(uniq_list[start_idx:end_idx], gf, protocol=pickle.HIGHEST_PROTOCOL)
                                            start_idx = end_idx
                                    print(f"GPU 去重完成，唯一边数: {len(uniq_list)}")
                            except Exception as e:
                                print("GPU 去重环节失败或不适用，继续。错误:", e)
                        
                    except Exception as e:
                        print(f"保存边列表时出错: {e}")
                        write_file += 1  # 仍然计数，避免无限循环

                # 随机小延迟以避免抢资源
                rnd_time = random.random() * 5
                time.sleep(rnd_time)
                
        except Exception as e:
            print(f"读取文件 {file_path} 时出错: {e}")
            write_file += 1  # 仍然计数，避免无限循环
                
    else:
        # 如果 log_file_txt 已存在，说明此 part 已经有人在处理或处理过
        # 检查是否所有文件都已处理完成（依据 finish 或 empty 文件）
        finish_pattern = os.path.join(log_folder, 'log_edge_part_*_finish.txt')
        empty_pattern = os.path.join(log_folder, 'log_edge_part_*_empty.txt')
        
        try:
            finished_files = [f for f in glob.glob(finish_pattern) if extract_id(f) is not None and extract_id(f) < len(curr_run_file_paths)]
            empty_files = [f for f in glob.glob(empty_pattern) if extract_id(f) is not None and extract_id(f) < len(curr_run_file_paths)]

            total_files = len(finished_files) + len(empty_files)

            if total_files >= len(curr_run_file_paths):
                print(f"{datetime.now()}: 所有文件处理完成!")
                break
        except Exception as e:
            print(f"检查完成状态时出错: {e}")

# 写入完成标记
try:
    with open("job_finish.txt", 'a', encoding='utf-8') as f:
        f.write(f'\n完成所有处理: {datetime.now()}\n')
    print("处理完成!")
except Exception as e:
    print(f"写入完成标记时出错: {e}")


=== 开始处理 ===
基础路径: D:/Impact4Cast-main/Impact4Cast-main/Impact4Cast-main/create_dynamic_edges/openalex-snaphot/data/works/
基础路径是否存在: True
找到的文件路径数量: 873
处理时间范围: 2022-12-20 00:00:00 到 2025-10-01 00:00:00
在时间范围内的文件数: 873
概念文件路径: d:\Impact4Cast-main\Impact4Cast-main\Impact4Cast-main\data_concept_graph\full_domain_concept.txt
概念文件是否存在: True
成功加载概念数量: 37960
正在构建 Aho-Corasick 自动机（可能需要一点时间）...
AC 自动机构建完成
开始处理 873 个文件...


KeyboardInterrupt: 

对概念共现边数据进行质量过滤，从所有概念关联边中筛选出只包含有实际引用（引用数不为零）的论文中的概念对。它会读取原始边数据文件，移除无人引用的论文产生的概念关联，将过滤后的高质量边数据保存到新的文件夹中，并统计显示过滤前后的数据量变化，确保后续分析基于有学术影响力的概念关系。

In [4]:
import os
import gzip
import pickle

# === 设置你的文件夹路径 ===
folder_path = r"D:\Impact4Cast-main\Impact4Cast-main\Impact4Cast-main\create_dynamic_edges\concept_pair"

# 输出路径（保存筛选后的结果）
output_folder = os.path.join(folder_path, "filtered")
os.makedirs(output_folder, exist_ok=True)

# 获取所有 .gz 文件
gz_files = [f for f in os.listdir(folder_path) if f.endswith(".gz")]
print(f"共找到 {len(gz_files)} 个文件。")

total_before = 0
total_after = 0

# 循环处理每个文件
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    output_path = os.path.join(output_folder, gz_file)

    try:
        with gzip.open(file_path, "rb") as f:
            edge_list = pickle.load(f)
    except Exception as e:
        print(f"❌ 无法读取 {gz_file}: {e}")
        continue

    total_before += len(edge_list)

    # 筛选：共被引次数不为 0
    filtered = [
        edge for edge in edge_list
        if len(edge) > 3 and isinstance(edge[3], (int, float)) and edge[3] != 0
    ]

    total_after += len(filtered)

    # 打印前几条看看
    print(f"\n📂 {gz_file}")
    print(f"原始记录: {len(edge_list)}  →  筛选后: {len(filtered)}")
    if filtered:
        for e in filtered[:5]:
            print(e)

        # 保存筛选结果
        with gzip.open(output_path, "wb") as f:
            pickle.dump(filtered, f)
    else:
        print("⚠️ 无符合条件的记录，跳过保存。")

# print("\n=== ✅ 全部完成 ===")
# print(f"总记录数: {total_before} → 保留 {total_after} （{total_after/total_before*100:.2f}%）")
# print(f"筛选后文件保存在: {output_folder}")

共找到 769 个文件。

📂 edge_part_002.gz
原始记录: 105  →  筛选后: 0
⚠️ 无符合条件的记录，跳过保存。

📂 edge_part_007.gz
原始记录: 1  →  筛选后: 1
[19177, 35411, 1922, 37, [{'year': 2022, 'cited_by_count': 2}, {'year': 2021, 'cited_by_count': 3}, {'year': 2020, 'cited_by_count': 2}, {'year': 2019, 'cited_by_count': 2}, {'year': 2018, 'cited_by_count': 2}, {'year': 2017, 'cited_by_count': 3}, {'year': 2016, 'cited_by_count': 3}, {'year': 2015, 'cited_by_count': 2}, {'year': 2014, 'cited_by_count': 2}, {'year': 2013, 'cited_by_count': 1}, {'year': 2012, 'cited_by_count': 2}]]

📂 edge_part_008.gz
原始记录: 36  →  筛选后: 36
[5715, 6618, 7259, 50, [{'year': 2023, 'cited_by_count': 2}, {'year': 2022, 'cited_by_count': 1}, {'year': 2021, 'cited_by_count': 4}, {'year': 2020, 'cited_by_count': 5}, {'year': 2019, 'cited_by_count': 6}, {'year': 2018, 'cited_by_count': 3}, {'year': 2017, 'cited_by_count': 5}, {'year': 2016, 'cited_by_count': 6}, {'year': 2015, 'cited_by_count': 6}, {'year': 2014, 'cited_by_count': 4}, {'year': 2013, 'cite